In [34]:
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import requests
import pandas as pd
import seaborn as sns
import re
import numpy as np

In [35]:

url = 'https://www.automobile.tn/fr/occasion'
nbre_page=2
n=1 #page 1
total_df = pd.DataFrame()

In [36]:
roads=[]
years=[]
boites=[]
transmissions=[]
horsepowers=[]
fuels=[]
nbre_portes=[]
cylindres=[]
items = []
marques = []
modeles = []
prices = []
links = []
features_tot=[]
features = []


In [40]:

for n in range(nbre_page):
    url = f'https://www.automobile.tn/fr/occasion/{n}'
    code_page = requests.get(url)
    soup = BeautifulSoup(code_page.text, 'html.parser')
    
    print(f"Scraping URL: {url}")

    container = soup.findAll(class_='occasion-item-v2')

    for item in container:
        # Extraire les caractéristiques avec gestion des valeurs manquantes
        road = item.find('li', class_='road')
        roads.append(road.text.strip() if road else np.nan)

        year = item.find('li', class_='year')
        years.append(year.text.strip() if year else np.nan)

        boite = item.find('li', class_='boite')
        boites.append(boite.text.strip() if boite else np.nan)

        transmission = item.find('li', class_='transmission')
        transmissions.append(transmission.text.strip() if transmission else np.nan)

        horsepower = item.find('li', class_='horsepower')
        horsepowers.append(horsepower.text.strip() if horsepower else np.nan)

        fuel = item.find('li', class_='fuel')
        fuels.append(fuel.text.strip() if fuel else np.nan)

        # Extraire la marque et le modèle
        noms = item.findAll('h2')
        for nom in noms:
            nom_car = nom.text.strip() if nom else np.nan
            if nom_car:
                marques.append(nom_car.split()[0])
                parts = nom_car.split()
                modeles.append(' '.join(parts[1:]))
            else:
                marques.append(np.nan)
                modeles.append(np.nan)

        # Extraire le prix
        price_tag = item.find('div', class_='price')
        price_final = price_tag.text.replace('DT', '').replace('\xa0', '').strip() if price_tag else np.nan
        prices.append(price_final)

        # Extraire le lien
        link_tag = item.find('a', class_='occasion-link-overlay', href=True)
        if link_tag:
            correct_link = f'https://www.automobile.tn/{link_tag["href"]}'
            links.append(correct_link)

    # Parcourir chaque lien pour les caractéristiques supplémentaires
    for link in links:
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'html.parser')
        container_a = soup.findAll('span', class_='spec-name')

        porte, cylindre = np.nan, np.nan
        for item in container_a:
            # Nombre de portes
            if item.text.strip() == 'Nombre de portes':
                porte = item.findNext('span').text.strip() if item.findNext('span') else np.nan
                nbre_portes.append(porte)

            # Cylindrée
            if item.text.strip() == 'Cylindrée':
                cylindre_text = item.findNext('span').text if item.findNext('span') else np.nan
                cylindre_number = re.search(r'\d+', cylindre_text).group() if cylindre_text else np.nan
                cylindres.append(cylindre_number)

        # Ajouter des valeurs manquantes si non trouvées
        if porte is np.nan:
            nbre_portes.append(np.nan)
        if cylindre is np.nan:
            cylindres.append(np.nan)

    # Vérification et création du DataFrame
    if len(prices) == len(marques) == len(modeles) == len(roads) == len(years) == len(boites) == len(transmissions) == len(horsepowers) == len(fuels) == len(nbre_portes) == len(cylindres):
        data = {
            'Marque': marques,
            'Modèle': modeles,
            'year': years,
            'road': roads,
            'boite': boites,
            'horsepower': horsepowers,
            'fuel': fuels,
            'nbre_portes': nbre_portes,
            'cylindrée': cylindres,
            'transmission': transmissions,
            'Prix': prices
        }
        df = pd.DataFrame(data)
        print(df)

    # Ajouter au DataFrame final
    total_df = pd.concat([total_df, df], ignore_index=True)

print(total_df)



Scraping URL: https://www.automobile.tn/fr/occasion/0
Scraping URL: https://www.automobile.tn/fr/occasion/1
           Marque                    Modèle  year        road        boite  \
0      Volkswagen                      Polo  2004  193 000 km     Manuelle   
1            Land  Rover Range Rover Evoque  2014  198 000 km  Automatique   
2   Mercedes-Benz                   GLA AMG  2017  150 000 km  Automatique   
3         Peugeot                       208  2015  110 000 km     Manuelle   
4             BMW    Série 3 coupé Kit M///  2012  152 000 km     Manuelle   
..            ...                       ...   ...         ...          ...   
67           Seat                     Ibiza  2019   98 000 km     Manuelle   
68         Toyota                      C-HR  2020   80 000 km  Automatique   
69  Mercedes-Benz              Classe E AMG  2021   98 000 km  Automatique   
70       Mahindra               KUV 100 K6+  2021   47 000 km     Manuelle   
71     Volkswagen                 

In [38]:

#la dataframe totale contenant les données scrapées de toutes les pages:
total_df.to_csv('../data/cars.csv', index=False)
print(total_df)


           Marque                    Modèle  year        road        boite  \
0      Volkswagen                      Polo  2004  193 000 km     Manuelle   
1            Land  Rover Range Rover Evoque  2014  198 000 km  Automatique   
2   Mercedes-Benz                   GLA AMG  2017  150 000 km  Automatique   
3         Peugeot                       208  2015  110 000 km     Manuelle   
4             BMW    Série 3 coupé Kit M///  2012  152 000 km     Manuelle   
5   Mercedes-Benz              Classe C AMG  2011  200 000 km  Automatique   
6          Toyota                C-HR Sport  2020  162 000 km  Automatique   
7            Seat                     Ibiza  2019   98 000 km     Manuelle   
8          Toyota                      C-HR  2020   80 000 km  Automatique   
9   Mercedes-Benz              Classe E AMG  2021   98 000 km  Automatique   
10       Mahindra               KUV 100 K6+  2021   47 000 km     Manuelle   
11     Volkswagen                    Tiguan  2021  123 000 km   